## Duplicate memory.

In this notebook, we will test that our ``agent`` is capable of handling duplicate memories.
Through its ``memory manager``, that will identify if a new memory is a duplicate by checking similarity based on context and tokens.
If a duplicate memory is found, the policy is to ignore the older ``memory`` in favor of the newer ``memory``.

In [1]:
from goodai.src.models import OpenSourceModel, Agent

In [2]:
model_name = "mistralai/Mixtral-8x7B-Instruct-v0.1"

In [3]:
model = OpenSourceModel(model_name)
agent = Agent(model)

[2024-03-03 22:42:33,056]:[root]:[INFO]: Connection to session database established.


First let's start with a clean ``session``, a session is a combinations of interactions with the ``agent``, these interactions are saved in a local ``sqlite`` database in order to preserve them, if we start a new session all information provided to the ``agent`` will be lost.

In [4]:
agent.new_session()

[2024-03-03 22:42:36,717]:[root]:[INFO]: Cleared session database.


Now, let's provide our ``agent`` with some informations.

In [5]:
agent.interact("I am going outside.")

'Great, have a nice time outside! Let me know if you need anything.'

In [6]:
agent.interact("I have returned home and am going to take a bath.")

'Got it! Have a nice bath.'

Now, let's check the content of our memory buffer.

In [7]:
agent.memory_manager.memory_buffer

[Memory(content=I am going outside., memory_type=new, timestamp=2024-03-03 22:42:37.975032, expiration=2024-05-02 22:42:37.975032),
 Memory(content=I have returned home and am going to take a bath., memory_type=new, timestamp=2024-03-03 22:42:38.828608, expiration=2024-05-02 22:42:38.828608)]

Now let's provide the same input to the ``agent``

In [8]:
agent.interact("I am going outside again.")

[2024-03-03 22:42:39,582]:[root]:[WARNING]: Duplicated memory found.


'Great, have a nice time outside!'

But the ``memory manager`` of the ``agent`` was able to detect that the last interaction is in fact a duplicate of a prior interaction, so the older memory is removed and the newer memory is inserted in its buffer, we can confirm this by checking its content.

In [9]:
agent.memory_manager.memory_buffer

[Memory(content=I have returned home and am going to take a bath., memory_type=new, timestamp=2024-03-03 22:42:38.828608, expiration=2024-05-02 22:42:38.828608),
 Memory(content=I am going outside again., memory_type=new, timestamp=2024-03-03 22:42:39.558270, expiration=2024-05-02 22:42:39.558270)]